# Hypothesis Testing

- Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

- Do some movie genres earn more revenue than others?

- Does the month of release affect revenue?

## Concanteate API Calls for years 2010 - 2019

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
import glob 
